In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import tensorflow as tf
from model import *

In [ ]:
# set cpu as the default device
tf.config.set_visible_devices([], 'GPU')

In [ ]:
from tensorflow.keras.layers import Dropout, Flatten, Conv2D, Input, BatchNormalization, Dense, MaxPooling2D, Activation, AveragePooling2D
   
input = Input(shape=(224,224,1))

# base model
conv1 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(input)
conv1 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(conv1)
conv1 = Dropout(0.1)(conv1)
mpool1 = MaxPooling2D()(conv1)

conv2 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(mpool1)
conv2 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(conv2)
conv2 = Dropout(0.2)(conv2)
mpool2 = MaxPooling2D()(conv2)

conv3 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(mpool2)
conv3 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(conv3)
conv3 = Dropout(0.3)(conv3)
mpool3 = MaxPooling2D()(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(mpool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(conv4)
conv4 = Dropout(0.4)(conv4)

out = tf.keras.layers.GlobalAveragePooling2D()(conv4)


model = tf.keras.Model(inputs=input, outputs=out)
model.load_weights("aug.hdf5", by_name=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# save the model tp a h5 file
model.save("aug.h5")

In [ ]:
# load the model from the saved file
model = tf.keras.models.load_model('aug.h5')

In [ ]:
data_dir = '/dmc/ml_storage/machine_learning/Final_ML_CBS/data/cohorts_5_aug/'
size = 224
image_generator = tf.keras.preprocessing.image.ImageDataGenerator()

datagen = image_generator.flow_from_directory(
        data_dir,
        target_size=(size, size),
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical',
        shuffle=True)

X = []
y = []
for image_batch, label_batch in tqdm(datagen):
    image = image_batch.flatten()
    image = image.reshape(size, size, 1)
    X.append(image)
    label = label_batch.flatten()
    y.append(label)
    if len(X) == len(datagen):
        break

X = np.array(X)
y = np.array(y)

In [ ]:
# predict the labels
X = model.predict(X)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)

In [ ]:
svm_model = svm.SVC(kernel='rbf', C=1, gamma=0.1)
svm_model.fit(X_train, y_train)

In [ ]:
# create a classification report
from sklearn.metrics import classification_report
y_pred = svm_model.predict(X_val)
print(classification_report(y_val, y_pred))